# Constants:

In [1]:
import warnings
from sage.symbolic.units import convert
from IPython.display import display
warnings.simplefilter('ignore')

%display latex

# declare units
V = units.electric_potential.volt
A = units.current.amp
ohm = units.resistance.ohm
m = units.length.meter
mm = units.length.millimeter

mt_voltage = 4 * V

# https://en.wikipedia.org/wiki/American_wire_gauge
resistivity_32_AWG = 0.5383 * ohm / m
resistivity_39_AWG = 2.729 * ohm / m

# Ferrocore Calculations:

In [2]:
l, r, N = var( 'l r N' )
u_r = var( 'u_r', latex_name="\mu_r" ) 
N_d = var( 'N_d', latex_name="N_d" )

ferrocore_turns = 1500
ferrocore_radius = 3.1 * mm
ferrocore_length = 70 * mm

# the length and radius are greater than 0 and the number of turns is more than 1
assume( l > 0 )
assume( r > 0 )
assume( N >= 1 )

# Demagnetization and amplification formulae used from:
# http://www.aerospades.com/uploads/3/7/3/2/37325123/cadre_torquers.pdf
demagnetization( l, r ) = 4 * ( ln( l / r ) - 1 ) / ( ( l / r )^2 - 4 * ln( l / r ) )

amplification( r, l, u_r ) = 1 + ( u_r - 1 ) / ( 1 + ( u_r - 1 ) * demagnetization( l, r ) )

current_ferrocore( r, N ) = mt_voltage / ( 2 * pi * r * N * resistivity_39_AWG )

dipole_ferrocore( r, l, N, u_r ) = pi * r^2 * N * current_ferrocore( N, r ) * amplification( r, l, u_r )

### Ferrocore current at standard parameters:

In [3]:
(current_ferrocore(
    r=ferrocore_radius,
    N=ferrocore_turns ).convert( A ) / A).n(30) * A

0.050167636*amp

### Ferrocore dipole at standard parameters:

In [4]:
# u_r = 15000
# http://www.edfagan.com/hiperco-50-coil.php

dipole_ferrocore( 
    r=ferrocore_radius,
    l=ferrocore_length, 
    N=ferrocore_turns,
    u_r=15000 ).convert( A * m^2 )

0.135198381808878*(amp*meter^2)

# Aircore calculations:

All outputs in A m<sup>2</sup>

In [5]:
l, w, N = var( 'l w N' )

# the length and width are positive, and the number of turns is greater than 1.
# Helps with symbolic optimization and debugging.
assume( l > 0 )
assume( w > 0 )
assume( N >= 1 )

aircore_length = 55*mm
aircore_width = 62*mm
aircore_turns = 820

# the current is the minimum between Ohm's law on 32 AWG wire and the max output of 50 mA
current_aircore( l, w, N ) = mt_voltage / ( ( 2 * l + 2 * w ) * N * resistivity_32_AWG )

# Basically, the dipole moment of the aircore is mu = NIA
dipole_aircore( l, w, N ) = N * current_aircore(l, w, N) * (l * w)

### Aircore current at standard parameters:

In [6]:
current_aircore(
    l=aircore_length,
    w=aircore_width,
    N=aircore_turns ).convert( A )

0.0387262907482388*amp

### Aircore dipole at standard parameters:

In [7]:
dipole_aircore(
    l=aircore_length,
    w=aircore_width,
    N=aircore_turns ).convert( A * m^2 )

0.108286454190225*(amp*meter^2)